In [64]:
import os

import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pyproj as proj
from datetime import datetime
import pandas as pd
from netCDF4 import Dataset  # pylint:disable=no-name-in-module
from shapely.geometry import Point

sf_type = 1
marker_size = 1

# Antarctica

In [65]:
ais_basins = gpd.read_file("aux_files/IMBIE_AIS_Basins/ANT_Basins_IMBIE2_v1.6.shp")
ais_basins = ais_basins.reset_index().rename(columns={"index": "basin_id"}).to_crs("epsg:4326")
ais_basins["basin_id"] = (ais_basins["basin_id"] + 1).astype(str)
ais_basins.head(18)

,basin_id,Regions,Subregion,geometry
0,1,Islands,None,"MULTIPOLYGON (((108.91949 -66.91047, 108.91933..."
1,2,West,H-Hp,"POLYGON ((-90.24588 -74.02074, -90.53000 -73.9..."
2,3,West,F-G,"POLYGON ((-144.36800 -75.62323, -144.36771 -75..."
3,4,East,E-Ep,"POLYGON ((-151.52291 -85.37243, -151.30170 -85..."
4,5,East,D-Dp,"POLYGON ((144.62895 -67.18320, 144.62074 -67.1..."
5,6,East,Cp-D,"POLYGON ((104.60697 -68.21552, 104.62590 -68.1..."
6,7,East,B-C,"POLYGON ((75.87282 -80.84156, 75.23614 -80.998..."
7,8,East,A-Ap,"POLYGON ((-5.68899 -72.94988, -5.77515 -72.721..."
8,9,East,Jpp-K,"POLYGON ((-47.77402 -81.92722, -47.66685 -81.9..."
9,10,West,G-H,"POLYGON ((-102.82001 -72.70919, -102.86595 -72..."


In [66]:
timedata_ais = pd.read_csv(
    "./processed_files/time_series_data_AIS.csv"
).sort_values("code")

max_v_ais = np.round(timedata_ais["Raw SEC"].abs().max(axis=None) + 0.05, 1)
timedata_ais["basin"] = timedata_ais["basin"].astype(str)
timedata_ais = pd.merge(
    timedata_ais,
    ais_basins[["basin_id", "Subregion"]],
    how="outer",
    left_on="basin",
    right_on="basin_id",
).drop(columns=["basin_id"])
timedata_ais.loc[timedata_ais["basin"] == "all", "Subregion"] = "All"
timedata_ais = timedata_ais.drop(columns=["basin", "code"])
timedata_ais.tail(12)

,period,midpoint,Raw SEC,Smooth SEC,dH,Subregion
6202,2017/10 - 2022/10,2020-04-30 00:57:32.343750,-0.006,-0.006,-1.794288,All
6203,2017/11 - 2022/11,2020-05-31 01:13:00.468750,-0.006,-0.006,-1.800504,All
6204,2017/12 - 2022/12,2020-06-30 00:52:47.578125,-0.006,-0.006,-1.806720,All
6205,2018/01 - 2023/01,2020-07-31 01:08:15.703125,-0.006,-0.006,-1.812943,All
6206,2018/02 - 2023/02,2020-08-29 01:16:31.406250,-0.006,-0.006,-1.819173,All
6207,2018/03 - 2023/03,2020-09-28 00:56:18.515625,-0.006,-0.006,-1.825403,All
6208,2018/04 - 2023/04,2020-10-29 00:11:46.640625,-0.006,-0.006,-1.831634,All
6209,2018/05 - 2023/05,2020-11-27 23:51:33.750000,-0.005,-0.005,-1.837061,All
6210,2018/06 - 2023/06,2020-12-29 00:07:01.875000,-0.005,-0.005,-1.841686,All
6211,2018/07 - 2023/07,2021-01-27 23:46:48.984375,-0.005,-0.005,-1.846310,All


In [67]:
import altair as alt
import vl_convert as vlc
import json
from IPython.display import Image

alt.data_transformers.enable("vegafusion")

click_basin_ais = alt.selection_point(fields=["Subregion"])

map_ais = (
    alt.Chart(
        ais_basins,
    )
    .mark_geoshape()
    .encode(
        color=alt.Color("Subregion:N").legend(None),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.2)),
        tooltip=["basin_id:N", "Subregion:N", "Regions:N"],
    )
    .project(type="stereographic", center=[-90, -180])
    .properties(width=400, height=400)
)

jchart1 = alt.JupyterChart(map_ais.add_params(click_basin_ais))
jchart1

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_12_store'…

In [68]:
sec_ais = (
    alt.Chart(timedata_ais, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X("midpoint:T", title="Time Period"),
        alt.Y(
            "Smooth SEC:Q",
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["Raw SEC", "period", "Subregion"],
    )
    .properties(width=900, height=200)
)

dh_ais = (
    alt.Chart(timedata_ais, title="Total SEC per basin")
    .mark_line()
    .encode(
        alt.X("midpoint:T", title="Time Period"),
        alt.Y(
            "dH:Q",
            title="Total elevation change (m)",
        ),
        opacity=alt.condition(click_basin_ais, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["dH", "period", "Subregion"],
    )
    .properties(width=900, height=200)
)

zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero).mark_rule(color="black", strokeDash=[1]).encode(y="zero:Q", size=alt.value(1))
)

line_chart_ais = (sec_ais + zero_rule) & (dh_ais + zero_rule)

jchart2 = alt.JupyterChart(line_chart_ais.add_params(click_basin_ais))
jchart2

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_12_store'…

In [69]:
chart = (line_chart_ais | map_ais).add_params(click_basin_ais)


jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_12_store'…

In [70]:
with open("ais_chart.json", "w+") as fp:
    fp.write(chart.to_json(format="vega"))

# Greenland

In [71]:
crs_new = ccrs.NorthPolarStereo(central_longitude=-40)
gis_basins = gpd.read_file(
    "aux_files/IMBIE_GIS_Basins/Greenland_Basins_PS_v1.4.2.shp"
).to_crs("EPSG:4326")
gis_basins = gis_basins.reset_index().rename(
    columns={"index": "basin_id", "SUBREGION1": "Subregion"}
)
gis_basins["basin_id"] = (gis_basins["basin_id"] + 1).astype(str)
gis_basins.head(10)

,basin_id,Subregion,NAME,GL_TYPE,geometry
0,1,CE,None,None,"POLYGON ((-32.84621 68.55410, -32.84621 68.554..."
1,2,CW,None,None,"POLYGON ((-51.20762 71.61893, -51.20951 71.618..."
2,3,NO,None,None,"POLYGON ((-71.46118 78.14071, -71.65543 78.133..."
3,4,NE,None,None,"POLYGON ((-22.65215 79.51328, -22.65259 79.511..."
4,5,NW,None,None,"POLYGON ((-52.26019 71.88370, -52.27234 71.886..."
5,6,SE,None,None,"POLYGON ((-39.14148 65.77665, -39.14072 65.779..."
6,7,SW,None,None,"POLYGON ((-50.24135 68.22278, -50.20214 68.234..."


In [72]:
click_basin_gis = alt.selection_point(fields=["Subregion"])

map_gis = (
    alt.Chart(
        gis_basins.to_crs(epsg="4326"),
    )
    .mark_geoshape()
    .encode(
        color=alt.Color(
            "Subregion:N",
            sort=["All", "NO", "NE", "NW", "CE", "CW", "SE", "SW"],
        ).legend(orient="right"),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.2)),
        tooltip=["basin_id:N", "Subregion:N"],
    )
    .project("stereographic", rotate=[0, 45, 20])
)

jchart4 = alt.JupyterChart(map_gis.add_params(click_basin_gis), debounce_wait=1000)
jchart4

JupyterChart(debounce_wait=1000.0, spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'n…

In [73]:
timedata_gis = pd.read_csv(
    "./processed_files/time_series_data_GIS.csv"
).sort_values("code")

max_v_gis = np.round(timedata_gis["Raw SEC"].abs().max(axis=None) + 0.05, 1)
# timedata_gis = timedata_gis.reset_index(drop=True).reset_index()
timedata_gis["basin"] = timedata_gis["basin"].astype(str)
timedata_gis = pd.merge(
    timedata_gis,
    gis_basins[["basin_id", "Subregion"]],
    how="outer",
    left_on="basin",
    right_on="basin_id",
).drop(columns="basin_id")
timedata_gis.loc[timedata_gis["basin"] == "all", "Subregion"] = "All"
timedata_gis = timedata_gis.drop(np.where(timedata_gis["basin"] == "0")[0], axis=0)
# timedata_gis.loc[timedata_gis["basin"] == "0", "Subregion"] = "Outside"

timedata_gis

,code,period,midpoint,basin,Raw SEC,Smooth SEC,dH,Subregion
0,1991-1996-07,1991/07 - 1996/07,1994-01-28 12:02:43.476562,1,-0.008,-0.008,0.000000,CE
1,1991-1996-08,1991/08 - 1996/08,1994-02-28 12:15:01.757812,1,-0.008,-0.008,-0.007654,CE
2,1991-1996-09,1991/09 - 1996/09,1994-03-31 12:55:09.960938,1,-0.008,-0.008,-0.015308,CE
3,1991-1996-10,1991/10 - 1996/10,1994-04-30 13:01:56.015625,1,-0.008,-0.006,-0.022963,CE
4,1991-1996-11,1991/11 - 1996/11,1994-05-31 13:14:14.296875,1,-0.008,-0.005,-0.030617,CE
...,...,...,...,...,...,...,...,...
2611,2018-2023-05,2018/05 - 2023/05,2020-11-27 23:51:33.750000,all,-0.099,-0.102,-13.764975,All
2612,2018-2023-06,2018/06 - 2023/06,2020-12-29 00:07:01.875000,all,-0.099,-0.100,-13.863513,All
2613,2018-2023-07,2018/07 - 2023/07,2021-01-27 23:46:48.984375,all,-0.099,-0.099,-13.962052,All
2614,2018-2023-08,2018/08 - 2023/08,2021-02-28 00:02:17.109375,all,-0.099,-0.099,-14.060591,All


In [76]:
line_gis = (
    alt.Chart(timedata_gis, title="Mean SEC per Basin")
    .mark_line()
    .encode(
        alt.X(
            "midpoint:T",
            axis=alt.Axis(tickMinStep=2),
            title="Time Period",
        ),
        alt.Y(
            "Smooth SEC:Q",
            scale=alt.Scale(domain=(-max_v_gis, max_v_gis)),
            title="Mean elevation change (m/year)",
        ),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.05)),
        color=alt.Color(
            "Subregion:N",
            sort=["All", "Outside basin", "NO", "NE", "NW", "CE", "CW", "SE", "SW"],
        ).legend(orient="right"),
        tooltip=["Smooth SEC", "period", "Subregion"],
    )
    .properties(width=900, height=300)
)

dh_gis = (
    alt.Chart(timedata_gis, title="Total SEC per basin")
    .mark_line()
    .encode(
        alt.X("midpoint:T", title="Time Period"),
        alt.Y(
            "dH:Q",
            title="Total elevation change (m)",
        ),
        opacity=alt.condition(click_basin_gis, alt.value(1), alt.value(0.05)),
        color=alt.Color("Subregion:N"),
        tooltip=["dH", "period", "Subregion"],
    )
    .properties(width=900, height=200)
)


zero = pd.DataFrame([{"zero": 0.0}])
zero_rule = (
    alt.Chart(zero)
    .mark_rule(color="black", strokeDash=[1])
    .encode(y="zero:Q", size=alt.value(1))
)

line_chart_gis = (line_gis + zero_rule) & (dh_gis + zero_rule)

jchart4 = alt.JupyterChart(line_chart_gis.add_params(click_basin_gis))
jchart4

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_13_store'…

In [77]:
chart = (line_chart_gis | map_gis).add_params(click_basin_gis)


jchart = alt.JupyterChart(chart)
jchart

JupyterChart(spec={'$schema': 'https://vega.github.io/schema/vega/v5.json', 'data': [{'name': 'param_13_store'…